In [1]:
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
from langchain_openai import ChatOpenAI

In [4]:
model = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0
)

In [5]:
from langchain_core.prompts import ChatPromptTemplate

In [6]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","you are a helpful assistant"),
        ("human", "{input}"),
    ]
)

In [7]:
msg = prompt.format_prompt(input="9.9 is grater than 9.11")

In [8]:
response = model.invoke(msg)

In [9]:
print(response.content)

Actually, 9.9 is not greater than 9.11. When comparing these two numbers, 9.9 is equivalent to 9.90, which is less than 9.11. Therefore, 9.11 is greater than 9.9.


## Tool을 사용해보기

In [10]:
from langchain.agents import tool

In [11]:
@tool
def compare_two_numbers(x: float, y: float) -> str:
    """
    compare float x and float y,
    return a blloean value True if x is grater than y, False if y is grager than X.
    """
    if x > y:
        return True
    else:
        return False

In [12]:
tools = [compare_two_numbers]

In [13]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

In [14]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "you are a helpful assistant"),
        ("human","{input}"),
        ("placeholder","{agent_scratchpad}")
        
    ]
)

In [15]:
agent=create_tool_calling_agent(model, tools, prompt)

In [16]:
agent_executor=AgentExecutor(agent=agent, tools=tools, verbose=True)

In [17]:
response = agent_executor.invoke(
    {
        "input" : "9.9 is greater than 9.11"
    }
)



> Entering new AgentExecutor chain...

Invoking: `compare_two_numbers` with `{'x': 9.9, 'y': 9.11}`


TrueYes, 9.9 is greater than 9.11.

> Finished chain.


In [18]:
print(response["output"])

Yes, 9.9 is greater than 9.11.


In [19]:
response = agent_executor.invoke(
    {
        "input" : "Which one is grater? 9.9 and 9.11"
    }
)



> Entering new AgentExecutor chain...

Invoking: `compare_two_numbers` with `{'x': 9.9, 'y': 9.11}`


True
Invoking: `compare_two_numbers` with `{'x': 9.11, 'y': 9.9}`


False9.11 is greater than 9.9.

> Finished chain.


In [20]:
print(response["output"])

9.11 is greater than 9.9.


tool calling agent 2

In [21]:
@tool
def find_phone_number(name: str) -> str:
    """
    주어진 사람들의 전화번호를 찾아서 반환합니다.
    전화번호를 찾지 못한 경우 None을 반환합니다.
    """
    phone_book = {
        "문벼리": "010-1234-1234",
        "양승렬": "010-2434-2434",
        "장동원": "010-4142-1424",
    }
    if name in phone_book:
        return phone_book[name]
    else:
        return None

In [22]:
tools = [find_phone_number]

In [23]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

In [24]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","you are a helpful assistant"),
        ("human","{input}"),
        ("placeholder","{agent_scratchpad}"),
    ]
)

In [25]:
agent = create_tool_calling_agent(model, tools, prompt)

In [26]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose = True)

In [27]:
response = agent_executor.invoke(
    {
        "input":"문벼리의 전화번호를 알려줘"
    }
)



> Entering new AgentExecutor chain...

Invoking: `find_phone_number` with `{'name': '문벼리'}`


010-1234-1234문벼리의 전화번호는 010-1234-1234입니다.

> Finished chain.


In [28]:
response

{'input': '문벼리의 전화번호를 알려줘', 'output': '문벼리의 전화번호는 010-1234-1234입니다.'}

In [29]:
agent_executor.verbose = False

In [30]:
response = agent_executor.invoke(
    {
        "input": "양승렬의 전화번호를 알려주세요"
    }
)

In [31]:
print(response["output"])

양승렬의 전화번호는 010-2434-2434입니다.


Multi Tools

In [32]:
@tool
def get_weather(location: str)-> str:
    """
    주어진 지역에 대한 날씨를 반환합니다.
    날씨 정보가 없는 경우 none을 반환합니다.
    """
    weather_data = {
        "서울": "맑음",
        "포항": "바람이 많이 붐",
        "제주도" : "흐림"
    }
    if location in weather_data:
        return weather_data[location]
    else:
        return None

In [33]:
tools = [find_phone_number, get_weather]

In [34]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", "you are a helpful assistant"
        ),
        (
            "human","{input}"
        ),
        (
            "placeholder","{agent_scratchpad}"
        ),
    ]
)

In [35]:
agent = create_tool_calling_agent(model, tools, prompt)

In [36]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [37]:
response = agent_executor.invoke(
    {
        "input":"문벼리의 전화번호를 알려주세요"
    }
)



> Entering new AgentExecutor chain...

Invoking: `find_phone_number` with `{'name': '문벼리'}`


010-1234-1234문벼리의 전화번호는 010-1234-1234입니다.

> Finished chain.


In [38]:
response

{'input': '문벼리의 전화번호를 알려주세요', 'output': '문벼리의 전화번호는 010-1234-1234입니다.'}

In [39]:
response = agent_executor.invoke(
    {
        "input":"오늘 포항 날씨를 알려주세요"
    }
)



> Entering new AgentExecutor chain...

Invoking: `get_weather` with `{'location': '포항'}`


바람이 많이 붐오늘 포항은 바람이 많이 붐니다. 날씨에 유의하세요!

> Finished chain.


In [40]:
response

{'input': '오늘 포항 날씨를 알려주세요', 'output': '오늘 포항은 바람이 많이 붐니다. 날씨에 유의하세요!'}

In [41]:
response = agent_executor.invoke(
    {
        "input":"문벼리에게 오늘 포항 날씨를 알려주고 싶은데, 문벼리의 전화번호와 포항 날씨를 알려줘"
    }
)



> Entering new AgentExecutor chain...

Invoking: `find_phone_number` with `{'name': '문벼리'}`


010-1234-1234
Invoking: `get_weather` with `{'location': '포항'}`


바람이 많이 붐문벼리의 전화번호는 010-1234-1234입니다. 포항의 날씨는 "바람이 많이 붑니다."입니다.

> Finished chain.


In [42]:
response

{'input': '문벼리에게 오늘 포항 날씨를 알려주고 싶은데, 문벼리의 전화번호와 포항 날씨를 알려줘',
 'output': '문벼리의 전화번호는 010-1234-1234입니다. 포항의 날씨는 "바람이 많이 붑니다."입니다.'}

Bind Tools with Model

In [47]:
msg = prompt.format_prompt(input="문벼리의 전화번호를 알려주세요.")

In [43]:
tools = [
    find_phone_number,
    get_weather
]
model_with_tools = model.bind_tools(tools)

In [48]:
response = model_with_tools.invoke(msg)

In [49]:
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_2Y3GjZBci0twztPh7SkCRKhu', 'function': {'arguments': '{"name":"문벼리"}', 'name': 'find_phone_number'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 116, 'total_tokens': 134, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': None, 'id': 'chatcmpl-BvIAF6Peemw6F6dLKl16M6Sz4R4Lb', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--a6c4fd8e-d287-488f-b797-245a1034507d-0', tool_calls=[{'name': 'find_phone_number', 'args': {'name': '문벼리'}, 'id': 'call_2Y3GjZBci0twztPh7SkCRKhu', 'type': 'tool_call'}], usage_metadata={'input_tokens': 116, 'output_tokens': 18, 'total_tokens': 134, 'input_token_details': {'audio

In [50]:
response.tool_calls

[{'name': 'find_phone_number',
  'args': {'name': '문벼리'},
  'id': 'call_2Y3GjZBci0twztPh7SkCRKhu',
  'type': 'tool_call'}]

In [51]:
type(response.tool_calls)

list

In [52]:
tool_call = response.tool_calls[0]

In [53]:
tool_call

{'name': 'find_phone_number',
 'args': {'name': '문벼리'},
 'id': 'call_2Y3GjZBci0twztPh7SkCRKhu',
 'type': 'tool_call'}

In [54]:
func = eval(tool_call["name"])

In [55]:
func.invoke("문벼리")

'010-1234-1234'